In [1]:
# Step 1:
import time

def log_calls(func):
    def wrapper(*args, **kwargs):
        print(f"Calling function '{func.__name__}' with args: {args} and kwargs: {kwargs}\n")
        result = func(*args, **kwargs)
        print(f"Function '{func.__name__}' returned: {result}\n")
        return result
    return wrapper

def timing(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()  
        result = func(*args, **kwargs)
        end_time = time.time() 
        print(f"Function '{func.__name__}' executed in {end_time - start_time:.6f} seconds\n")
        return result
    return wrapper

def require_admin(func):
    def wrapper(user, *args, **kwargs):
        if not getattr(user, 'is_admin', False):
            raise PermissionError("You need admin privileges to perform this action.\n")
        return func(user, *args, **kwargs)
    return wrapper

class User:
    def __init__(self, name, is_admin=False):
        self.name = name
        self.is_admin = is_admin

# Example usage 
@log_calls
def add(a, b):
    return a + b

add(35, 7)
add(4, 6)

@timing
def slow_function(duration):
    time.sleep(duration)
    return "Done"

slow_function(2)

@require_admin
def delete_user(admin_user, target_user):
    print(f"Admin '{admin_user.name}' deleted user '{target_user.name}'.")

admin = User("AdminUser", is_admin=True)
regular_user = User("RegularUser")

delete_user(admin, regular_user)

try:
    delete_user(regular_user, admin)
except PermissionError as e:
    print(f"Permission denied: {e}")

@timing
@log_calls
def multiply(a, b):
    return a * b

multiply(25,6)



Calling function 'add' with args: (35, 7) and kwargs: {}

Function 'add' returned: 42

Calling function 'add' with args: (4, 6) and kwargs: {}

Function 'add' returned: 10

Function 'slow_function' executed in 2.001231 seconds

Admin 'AdminUser' deleted user 'RegularUser'.
Permission denied: You need admin privileges to perform this action.

Calling function 'multiply' with args: (25, 6) and kwargs: {}

Function 'multiply' returned: 150

Function 'wrapper' executed in 0.000000 seconds



150

In [9]:
# Step 2:
def call_counter(func):
    count = 0
    
    def wrapper(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"{func.__name__} has been called {count} times.")
        return func(*args, **kwargs)
    
    return wrapper
    
@call_counter
def greet(name):
    print(f"Welcome, {name}!")

# Call the function multiple times
greet("Nagaraj")
greet("Guru")
greet("Suman")


greet has been called 1 times.
Welcome, Nagaraj!
greet has been called 2 times.
Welcome, Guru!
greet has been called 3 times.
Welcome, Suman!


In [11]:
# Step 3:
class EnforceRequirements(type):
    def __init__(cls, name, bases, cls_dict):
        super().__init__(name, bases, cls_dict)

        # Enforce that the class has a 'required_method'
        if 'required_method' not in cls_dict or not callable(cls_dict['required_method']):
            raise TypeError(f"{name} class must define a 'required_method' method.")

        # Enforce that the class has a 'required_attribute'
        if 'required_attribute' not in cls_dict:
            raise TypeError(f"{name} class must have a 'required_attribute' attribute.")

class MyClass(metaclass=EnforceRequirements):
    required_attribute = "This is required."

    def required_method(self):
        print("This method is required.")

# The following class will fail due to missing requirements
try:
    class InvalidClass(metaclass=EnforceRequirements):
        pass
except TypeError as e:
    print(e)

# Example of using MyClass
obj = MyClass()
obj.required_method()
print(obj.required_attribute)



InvalidClass class must define a 'required_method' method.
This method is required.
This is required.


In [13]:
# Step 4:
class RegistryMeta(type):
    registry = {}

    def __new__(cls, name, bases, class_dict):
        new_class = super().__new__(cls, name, bases, class_dict)
        cls.registry[name] = new_class
        return new_class

    @classmethod
    def get_class_by_name(cls, name):
        return cls.registry.get(name)
        
    @classmethod
    def get_classes_by_type(cls, base_type):
        return [klass for klass in cls.registry.values() if issubclass(klass, base_type) and klass is not base_type]


class BaseRegisteredClass(metaclass=RegistryMeta):
    pass


# Example classes 
class Scholarship(BaseRegisteredClass):
    def scholarship(self):
        return "Test pass the students"


class Girls(BaseRegisteredClass):
    def present_students(self):
        return "First five students"


class Boys(BaseRegisteredClass):
    def pass_students(self):
        return "First four students"


# Example Usage
if __name__ == "__main__":
    scholarship_class = RegistryMeta.get_class_by_name('Scholarship')
    if scholarship_class:  
        scholarship_instance = scholarship_class()  
        print(f"Lookup by name 'Scholarship': {scholarship_instance.scholarship()}")
        
    registered_classes = RegistryMeta.get_classes_by_type(BaseRegisteredClass)
    for registered_class in registered_classes:
        instance = registered_class()
        if hasattr(instance, 'present_students'):
            print(f"Instance of '{registered_class.__name__}': {instance.present_students()}")
        elif hasattr(instance, 'pass_students'):
            print(f"Instance of '{registered_class.__name__}': {instance.pass_students()}")

    # List all registered classes
    print("Registered classes:", RegistryMeta.registry)


Lookup by name 'Scholarship': Test pass the students
Instance of 'Girls': First five students
Instance of 'Boys': First four students
Registered classes: {'BaseRegisteredClass': <class '__main__.BaseRegisteredClass'>, 'Scholarship': <class '__main__.Scholarship'>, 'Girls': <class '__main__.Girls'>, 'Boys': <class '__main__.Boys'>}
